# ⚡ Apache Spark RDD Fundamentals

**Phase 5: Big Data Processing (RDD Module 1)**

**Prerequisites**: Basic Scala collections and functional programming

---

## 🔗 Spark RDD Architecture

Understanding Resilient Distributed Datasets (RDD) - the fundamental abstraction of Apache Spark.

In [ ]:
// Import Spark dependencies (for notebook environment)
// In a real environment, this would run on Spark cluster
import scala.collection.parallel.ParSeq

// RDD characteristics simulation in Scala collections
case class RDD[T](data: ParSeq[T], partitions: Int) {
  
  // RDD properties
  lazy val partitionsData = data.grouped(data.size / partitions + 1).toList
  
  def map[U](f: T => U): RDD[U] = {
    copy(data = data.map(f))
  }
  
  def filter(predicate: T => Boolean): RDD[T] = {
    copy(data = data.filter(predicate))
  }
  
  def collect(): List[T] = data.toList
  
  // Simulate distributed operations
  def reduce(f: (T, T) => T): Option[T] = {
    if (data.isEmpty) None
    else Some(data.reduce(f))
  }
  
  def count(): Long = data.size.toLong
  
  def distinct(): RDD[T] = {
    copy(data = data.distinct)
  }
}

// Companion object for RDD creation
object RDD {
  def fromSeq[T](seq: Seq[T], partitions: Int = 4): RDD[T] = {
    RDD(seq.par, partitions)
  }

  def range(start: Long, end: Long, partitions: Int = 4): RDD[Long] = {
    val data = (start until end).par
    RDD(data, partitions)
  }
}

// Test RDD creation and basic operations
val numbersRDD = RDD.fromSeq((1 to 20).toList)
println(s"RDD created with ${numbersRDD.count()} elements in ${numbersRDD.partitions} partitions")
println(s"Partition data: ${numbersRDD.partitionsData.map(_.size)}")
println()

## 🔄 RDD Transformations vs Actions

Understanding the fundamental difference between transformations and actions in Spark.

In [ ]:
// RDD Operation Categories
case class RDDAnalysis[T](rdd: RDD[T]) {
  
  // Transformations (lazy) - return new RDD
  def mapTransform[U](f: T => U): RDD[U] = {
    println(s"[TRANSFORMATION] mapTransform applying function")
    rdd.map(f)
  }
  
  def filterTransform(predicate: T => Boolean): RDD[T] = {
    println(s"[TRANSFORMATION] filterTransform applying predicate")
    rdd.filter(predicate)
  }
  
  def flatMapTransform[U](f: T => Iterable[U]): RDD[U] = {
    println(s"[TRANSFORMATION] flatMapTransform applying function")
    RDD(rdd.data.flatMap(f), rdd.partitions)
  }
  
  // Actions (eager) - compute and return results
  def countAction(): Long = {
    println(s"[ACTION] countAction collecting ${rdd.data.size} elements")
    rdd.count()
  }
  
  def collectAction(): List[T] = {
    println(s"[ACTION] collectAction returning all ${rdd.data.size} elements")
    rdd.collect()
  }
  
  def reduceAction(f: (T, T) => T): Option[T] = {
    println(s"[ACTION] reduceAction combining ${rdd.data.size} elements")
    rdd.reduce(f)
  }
}

// Demonstrate lazy vs eager evaluation
val originalRDD = RDD.fromSeq(List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10))
val analyzer = RDDAnalysis(originalRDD)

println("=== LAZY TRANSFORMATIONS (no computation yet) ===")
val transformedRDD = analyzer
  .mapTransform(_ * 2)
  .filterTransform(_ > 10)
  .mapTransform(_.toString)

println("\n=== EAGER ACTION (triggers computation) ===")
val result = analyzer.filterTransform(_ > 5).collectAction()
println(s"Final result: $result")
println()

## ⚡ Advanced RDD Operations

Key operations that make Spark RDDs powerful for big data processing.

In [ ]:
// Word count example - classic big data problem
def wordCountRDD(texts: Seq[String]): RDD[(String, Int)] = {
  val textRDD = RDD.fromSeq(texts)
  
  val wordCounts = textRDD
    .flatMapTransform(text => text.toLowerCase.split("\\W+"))
    .filterTransform(_.nonEmpty)
    .mapTransform(word => (word, 1))
    .collectAction()
    .groupBy(_._1)
    .mapValues(_.map(_._2).sum)
  
  // Convert back to RDD-like structure
  RDD.fromSeq(wordCounts.toList)
}

// Set operations with RDDs
def rddIntersection[T](rdd1: RDD[T], rdd2: RDD[T]): RDD[T] = {
  val set1 = rdd1.collectAction().toSet
  RDD.fromSeq(rdd2.collectAction().filter(set1.contains))
}

def rddUnion[T](rdd1: RDD[T], rdd2: RDD[T]): RDD[T] = {
  RDD.fromSeq(rdd1.collectAction() ++ rdd2.collectAction())
}

def rddDistinct[T](rdd: RDD[T]): RDD[T] = {
  RDD.fromSeq(rdd.collectAction().distinct)
}

// Sample data and operations
val sampleTexts = Seq(
  "Hello world hello Scala",
  "Scala is amazing Scala programming",
  "Spark and Scala make big data easy",
  "Hello Spark world"
)

val wordCounts = wordCountRDD(sampleTexts)
println("Word Count Results:")
wordCounts.collectAction()
  .sortBy(-_._2)  // Sort by frequency descending
  .foreach { case (word, count) =>
    println(f"  $word%15s : $count")
  }

// Set operations example
val rddA = RDD.fromSeq(Seq(1, 2, 3, 4, 5))
val rddB = RDD.fromSeq(Seq(3, 4, 5, 6, 7))
val rddC = RDD.fromSeq(Seq(1, 2, 8, 9, 10))

println("\nSet Operations:")
println(s"A ∩ B (intersection): ${rddIntersection(rddA, rddB).collectAction().sorted}")
println(s"A ∪ B (union):        ${rddDistinct(rddUnion(rddA, rddB)).collectAction().sorted}")
println()

## 🧩 RDD Persistence & Caching

Understanding when and how to cache RDDs for performance optimization.

In [ ]:
// RDD Caching simulation (in real Spark, this manages memory/disk persistence)
import scala.collection.mutable.Map

case class CachedRDD[T](
  rdd: RDD[T],
  storageLevel: StorageLevel = StorageLevel.NONE
) {
  private var cachedResult: Option[List[T]] = None
  private var accessCount = 0
  private var cacheHits = 0
  
  def cache(level: StorageLevel = StorageLevel.MEMORY_AND_DISK): CachedRDD[T] = {
    println(s"Caching RDD with storage level: $level")
    var cost = "low"
    if (level == StorageLevel.MEMORY_AND_DISK) cost = "medium"
    else if (level == StorageLevel.DISK_ONLY) cost = "high"
    println(s"Cache operation cost: $cost")
    this.copy(storageLevel = level)
  }
  
  def unpersist(): CachedRDD[T] = {
    cachedResult = None
    println("Unpersisting RDD")
    this.copy(storageLevel = StorageLevel.NONE)
  }
  
  def collectOptimized(): List[T] = {
    accessCount += 1
    
    if (storageLevel != StorageLevel.NONE && cachedResult.isDefined) {
      cacheHits += 1
      println(s"Cache hit! Using cached data ($cacheHits/$accessCount access ratio)")
      cachedResult.get
    } else {
      println(s"Cache miss! Computing and ${if (storageLevel != StorageLevel.NONE) "caching" else "not caching"} result")
      val result = rdd.collect()
      if (storageLevel != StorageLevel.NONE) {
        cachedResult = Some(result)
      }
      result
    }
  }
  
  def getStorageLevel: StorageLevel = storageLevel
}

// Storage level enumeration
sealed trait StorageLevel
object StorageLevel {
  case object NONE extends StorageLevel
  case object DISK_ONLY extends StorageLevel
  case object MEMORY_ONLY extends StorageLevel
  case object MEMORY_AND_DISK extends StorageLevel
  
  def toString(level: StorageLevel): String = level match {
    case NONE => "NONE"
    case DISK_ONLY => "DISK_ONLY"
    case MEMORY_ONLY => "MEMORY_ONLY"
    case MEMORY_AND_DISK => "MEMORY_AND_DISK"
  }
}

// Demonstrate caching benefits
val expensiveComputationRDD = RDD.fromSeq((1 to 10000).map(n => {
  // Simulate expensive calculation
  Thread.sleep(1)  // 1ms delay per element (would be computational work)
  n * n
}))

val cachedRDD = CachedRDD(expensiveComputationRDD).cache(StorageLevel.MEMORY_ONLY)

println("Cached RDD Performance Test:")

// First access - cache miss
val start1 = System.nanoTime()
val result1 = cachedRDD.collectOptimized().take(5)
val time1 = System.nanoTime() - start1
println(f"First access:  ${time1/1e6}%.1f ms")

// Second access - cache hit
val start2 = System.nanoTime()
val result2 = cachedRDD.collectOptimized().take(5)
val time2 = System.nanoTime() - start2
println(f"Second access: ${time2/1e6}%.1f ms (${time1/time2}%.1fx faster)")

// Third access - cache hit again
val start3 = System.nanoTime()
val result3 = cachedRDD.collectOptimized().take(5)
val time3 = System.nanoTime() - start3
println(f"Third access:  ${time3/1e6}%.1f ms")

cachedRDD.unpersist()
println()

## 🏗️ RDD Lineage & Fault Tolerance

Understanding how Spark maintains data lineage for automatic fault recovery.

In [ ]:
// RDD Lineage tracking (simplified DAG representation)
case class RDDOperation(
  operationType: String,
  description: String,
  inputRDDs: List[String] = Nil,
  parameters: Map[String, Any] = Map.empty
)

case class RDDLineage(
  currentRDD: String,
  operations: List[RDDOperation],
  dependencies: List[String]
) {
  
  def addOperation(op: RDDOperation): RDDLineage = {
    this.copy(
      operations = operations :+ op,
      dependencies = op.inputRDDs ::: dependencies
    )
  }
  
  def printLineage(): Unit = {
    println(s"Lineage for RDD: $currentRDD")
    println(s"Dependencies: ${dependencies.mkString(", ")}")
    println("Operations:")
    operations.zipWithIndex.foreach { case (op, idx) =>
      println(f"  ${idx+1}. ${op.operationType}%12s : ${op.description}")
      if (op.parameters.nonEmpty) {
        println(f"      Params: ${op.parameters.mkString(", ")}")
      }
    }
  }
  
  def lineageDepth: Int = operations.size
  def isNarrowDependency(index: Int): Boolean = {
    // Simplified - narrow if operation affects single partition
    operations(index).operationType match {
      case "map" | "filter" | "flatMap" => true
      case "reduceByKey" | "groupByKey" => false
      case _ => true
    }
  }
}

// Demonstrate RDD lineage tracking
var lineage = RDDLineage("numbersRDD", Nil, Nil)

lineage = lineage.addOperation(
  RDDOperation("parallelize", "Create parallel collection", Nil, Map("elements" -> 1000))
)

lineage = lineage.addOperation(
  RDDOperation("map", "Multiply by 2", List("numbersRDD"), Map("function" -> "_ * 2"))
)

lineage = lineage.addOperation(
  RDDOperation("filter", "Keep even numbers", List(lineage.currentRDD), Map("predicate" -> "_ % 2 == 0"))
)

lineage = lineage.addOperation(
  RDDOperation("reduce", "Sum all elements", List(lineage.currentRDD), Map("combiner" -> "_ + _"))
)

println("RDD Lineage Analysis:")
lineage.printLineage()
println(f"\nLineage depth: ${lineage.lineageDepth}")
println(s"Dependency types: ${lineage.operations.indices.map(i =>
  s"Op${i+1}=${if (lineage.isNarrowDependency(i)) "narrow" else "wide"}").mkString(", ")}")
println()

## 🎯 RDD Best Practices & Performance

### **Key RDD Optimization Principles:**

1. **Minimize object creation in transformations**
2. **Avoid unnecessary shuffling with wide dependencies**
3. **Cache intermediate results when needed**
4. **Prefer lazy evaluation benefits**
5. **Use appropriate storage levels**

### **When to Use RDDs vs DataFrames:**

| Use Case | RDD | DataFrame |
|----------|-----|-----------|
| **Control** | Full control over everything | Limited to DF operations |
| **Type Safety** | Compile-time type checking | Runtime schema validation |
| **Performance** | Can optimize at low level | Catalyst optimizer works |
| **Learning Curve** | Steeper (functional programming) | Easier (SQL-like) |
| **ML Pipelines** | Requires MLlib | Integrated ML capabilities |

### **RDD vs DataFrame Code Comparison:**

**RDD Way:**
```scala
val rddResult = sc.textFile("data.txt")
  .map(line => line.split(","))
  .filter(fields => fields(0).toInt > 100)
  .map(fields => (fields(1), fields(2).toDouble))
  .reduceByKey(_ + _)
```

**DataFrame Way:**
```scala
val dfResult = spark.read.csv("data.txt")
  .filter(col("field0") > 100)
  .groupBy("field1")
  .sum("field2")
```

### **Performance Tips:**

1. **Partition Optimization**
   - Avoid too many partitions (200-1000 is good)
   - Avoid too few partitions (underutilizes cluster)
   - Consider data locality

2. **Memory Management**
   - Use `persist(StorageLevel.MEMORY_AND_DISK_SER)` for large objects
   - Use `unpersist()` when data no longer needed
   - Be careful with closures (they capture whole objects)

3. **Shuffle Optimization**
   - Minimize wide transformations
   - Use `coalesce()` to reduce partitions after filtering
   - `repartition()` strategically

### **Debugging RDD Operations:**

```scala
// Inspect RDD partitions
rdd.mapPartitionsWithIndex((index, iter) => {
  println(s"Partition $index has ${iter.size} elements")
  iter
}).collect()

// Trace RDD lineage
rdd.toDebugString

// Sample data for debugging
rdd.take(10).foreach(println)
```

### **RDD Fault Tolerance Strategy:**

When a partition is lost:
1. Spark identifies lost partition
2. Recreates partition from lineage
3. Only lost partition is recomputed
4. Other partitions continue working

```scala
// Simulate fault tolerance
val reliableRDD = sc.parallelize(1 to 1000, 10)
  .map(n => n * 2)
  .persist(StorageLevel.MEMORY_AND_DISK)
  .filter(_ > 100)  // Later operations can fail
  .cache()          // Intermediate result cached
```

### **RDD Limitations:**

1. **No Automatic Optimization** - Unlike DataFrames, no Catalyst optimizer
2. **Type Safety Risk** - Can have runtime exceptions from type mismatches
3. **Manual Optimization** - Developer must manually optimize execution
4. **Verbose Code** - Functional operations can be more verbose than SQL
5. **Debugging Complexity** - Harder to debug complex transformations

### **When RDDs Excel:**

- **Custom Partitioner Logic** - Need specific data distribution
- **Low-Level Control** - Precise memory management needed
- **Binary Data Processing** - Working with serialized objects
- **Custom Algorithm Implementation** - Parallel graph algorithms, machine learning
- **Real-Time Processing** - Integration with streaming systems

**Next: DataFrame API and Spark SQL for more structured processing**